In [1]:
from utils.general import scale_coords, non_max_suppression, xyxy2xywh, xywhn2xyxy, xyxy2xywhn
import coremltools
import torch
import numpy as np
import random
import os
import PIL.Image
import PIL.ImageDraw 
import shutil
import time
from tqdm import tqdm

scikit-learn version 0.24.1 is not supported. Minimum required version: 0.17. Maximum required version: 0.19.2. Disabling scikit-learn conversion API.
TensorFlow version 2.5.0 detected. Last version known to be fully compatible is 2.3.1 .
Keras version 2.5.0 detected. Last version known to be fully compatible of Keras is 2.2.4 .
Adding op 'pow' of type pow
Adding op 'pow_y_0' of type const
Adding op 'mul_1' of type mul
Adding op 'mul_1_x_0' of type const
Adding op 'add' of type add
Adding op 'mul_2' of type mul
Adding op 'mul_2_x_0' of type const
Adding op 'tanh' of type tanh
Adding op 'add_1' of type add
Adding op 'add_1_x_0' of type const
Adding op 'mul' of type mul
Adding op 'mul_x_0' of type const
Adding op 'mul_3' of type mul
Adding op 'pow' of type pow
Adding op 'pow_y_1' of type const
Adding op 'mul_1' of type mul
Adding op 'mul_1_x_1' of type const
Adding op 'add' of type add
Adding op 'mul_2' of type mul
Adding op 'mul_2_x_1' of type const
Adding op 'tanh' of type tanh
Adding 

In [3]:
# Parameters and Global variables
MODE = 'debug'
SAVE_IMG = True
VIEW_IMG = False
SAVE_TXT = True
ENSEMBLE = False
CAT_NAMES = ['Screw', 'unknown']

# Anchor box can be checked in pytorch model
ANCHORS = ([2.375,3.375, 5.5,5, 4.75,11.75], 
           [6,4.25, 5.375,9.5, 11.25,8.5625], 
           [4.375,9.40625, 9.46875,8.25, 7.43750,16.93750],
           [6.81250,9.60938, 11.54688,5.93750, 14.45312,12.37500])
# stide can be check in pytorch model
stride = [8, 16, 32, 64]
# target size of input image (width, height)
IMG_SIZE = (2304, 3072)
# confidence threshold
conf_thres = .2


# Params and global variables that should be kept as it is
COLORS = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(CAT_NAMES))]
PATH = "./"
nc = len(CAT_NAMES)
nl = len(ANCHORS)
na = len(ANCHORS[0]) // 2
no = nc + 5
# initiate grid and anchor_grid
grid = [torch.zeros(1)] * nl
a = torch.tensor(ANCHORS).float().view(nl, -1, 2)
anchor_grid = [torch.zeros(1)] * nl


COREML_MODEL = ["/Users/zhenyu/Box/MLProject:IphoneAOI/weights/yolov5l6_3072*2304_20211116/weights/best.mlmodel",
                "/Users/zhenyu/Box/MLProject:IphoneAOI/weights/yolov5l6_3072*2304_20211124/weights/best.mlmodel"]
IMAGE_FOLDER = "/Users/zhenyu/Desktop/val/"
OUT_FOLDER = "/Users/zhenyu/Desktop/test/"

In [4]:
def make_grid(nx=20, ny=20, i=0):
    yv, xv = torch.meshgrid([torch.arange(ny), torch.arange(nx)])
    grid = torch.stack((xv, yv), 2).view((1, 1, ny, nx, 2)).float()
    anchor_grid = (a[i] * stride[i]).view((1, na, 1, 1, 2)).expand(1, na, ny, nx, 2).float()
    return grid, anchor_grid

def resize_image(source_image):
    background = PIL.Image.new('RGB', IMG_SIZE, "black")
    source_image.thumbnail(IMG_SIZE)
    (w, h) = source_image.size
    background.paste(source_image, (int((IMG_SIZE[0] - w) / 2), int((IMG_SIZE[1] - h) / 2 )))
    return background

In [5]:
def eval(image, model, file_name):
    resized = resize_image(image.copy())

    predictions = model.predict({'image': resized})

    z = []  # inference output
    x = []
    for head in ['var_1763', 'var_1778', 'var_1793', 'var_1808']:
        x.append(torch.Tensor(predictions[head]))

    for i in range(nl):
        bs, _, ny, nx, _ = x[i].shape

        # if grid[i].shape[2:4] != x[i].shape[2:4]:
        grid[i], anchor_grid = make_grid(nx, ny, i)

        y = x[i].sigmoid()
        y[..., 0:2] = (y[..., 0:2] * 2. - 0.5 + grid[i]) * stride[i]  # xy
        y[..., 2:4] = (y[..., 2:4] * 2) ** 2 * anchor_grid  # wh
        z.append(y.view(bs, -1, no))
    
    pred = torch.cat(z, 1)
    
    return pred

In [6]:
def debug(model_list):
    if os.path.exists(OUT_FOLDER):
        shutil.rmtree(OUT_FOLDER)
    os.makedirs(OUT_FOLDER)
    
    time_tracker = {}
    time_sum = 0
    for img_path in tqdm(os.listdir(IMAGE_FOLDER)):
        if img_path.endswith(".jpg") and not img_path.startswith('.'):
            image = PIL.Image.open(os.path.join(IMAGE_FOLDER, img_path))
            draw = PIL.ImageDraw.Draw(image)
            pred = torch.tensor([])
            t0 = time.time()
            for model in model_list:
                pred = torch.cat((pred, eval(image, model, img_path)), 1)
            nms = non_max_suppression(pred, conf_thres, .3, classes=None, agnostic=False)[0]
            label=[]
            for *xyxy, _, cls in nms:
                if SAVE_TXT:
                    xywh = xyxy2xywhn(torch.tensor(xyxy).view(1, 4), w=IMG_SIZE[0], h=IMG_SIZE[1]).view(-1).tolist()
                    label.append(('%g ' * 5 + '\n') % (cls, *xywh))
                if SAVE_IMG:
                    draw.rectangle(np.array(torch.tensor(xyxy).view(2,2)*1.3125), outline='red', width=6)
            if SAVE_TXT:
                with open(os.path.join(OUT_FOLDER, '{}.txt'.format(img_path[:-4])), 'a') as f:
                    for line in label:
                        f.write(line)
            if SAVE_IMG:
                image.save(os.path.join(OUT_FOLDER, '{}.jpg'.format(img_path[:-4])))
            delta_t = time.time() - t0
            time_tracker[img_path] = delta_t
        break
    for key, item in time_tracker.items():
        print('{} takes {} seconds'.format(key, item))
        time_sum += item
    print('Averange process time is {}'.format(time_sum/len(time_tracker)))

In [7]:
model_list = [coremltools.models.model.MLModel(model) for model in COREML_MODEL]

In [8]:
debug(model_list)

  0%|          | 0/70 [00:03<?, ?it/s]

16EWQV5RIM.jpg takes 3.5988080501556396 seconds
Averange process time is 3.5988080501556396


In [44]:
det.shape

torch.Size([19, 6])

In [36]:
torch.cat(z, 1).shape

torch.Size([1, 440640, 7])

In [11]:
((torch.cat(z, 1))[:,:,4] > 0.5).shape

torch.Size([1, 440640])

In [8]:
filtered = torch.cat(z, 1)[torch.where(torch.cat(z, 1)[:,:,4]>0.2)]

In [12]:
filtered[:,-1].shape

torch.Size([238])

In [13]:
torch.nn.Sigmoid(filtered[:,1])

TypeError: __init__() takes 1 positional argument but 2 were given

In [21]:
det = torch.cat((det, det))

In [46]:
test = torch.cat((torch.cat(z, 1), torch.cat(z, 1)), 1)

In [47]:
test.shape

torch.Size([1, 881280, 7])

In [48]:
pred_det = non_max_suppression(test, 0.2, .3, classes=None, agnostic=False)

In [50]:
pred_det[0].shape

torch.Size([19, 6])

In [22]:
det

tensor([[1.46316e+03, 1.47392e+03, 1.49889e+03, 1.51176e+03, 8.58536e-01, 0.00000e+00],
        [8.21176e+02, 2.74568e+02, 8.55816e+02, 3.11257e+02, 8.56946e-01, 0.00000e+00],
        [1.33926e+03, 6.36934e+02, 1.37504e+03, 6.73071e+02, 8.49743e-01, 0.00000e+00],
        [1.33596e+03, 1.41064e+03, 1.36180e+03, 1.43827e+03, 8.44309e-01, 0.00000e+00],
        [9.03651e+02, 1.73896e+02, 9.32065e+02, 2.03853e+02, 8.43017e-01, 0.00000e+00],
        [8.13361e+02, 4.52000e+02, 8.45583e+02, 4.87027e+02, 8.40511e-01, 0.00000e+00],
        [1.43460e+03, 2.23147e+03, 1.46911e+03, 2.26926e+03, 8.38835e-01, 0.00000e+00],
        [1.34529e+03, 1.32787e+03, 1.37672e+03, 1.36156e+03, 8.38572e-01, 0.00000e+00],
        [7.52156e+02, 1.79194e+02, 7.73399e+02, 2.03658e+02, 8.37310e-01, 0.00000e+00],
        [1.14954e+03, 7.42505e+02, 1.17441e+03, 7.68922e+02, 8.33452e-01, 0.00000e+00],
        [1.29396e+03, 1.72329e+03, 1.33008e+03, 1.76205e+03, 8.31392e-01, 0.00000e+00],
        [1.21800e+03, 1.25558e+0

In [15]:
import torchvision

In [17]:
nms = torchvision.ops.batched_nms(filtered[:, :4], filtered[:,4], torch.zeros(238), 0.3)

In [16]:
nms_2 = torchvision.ops.nms(filtered[:, :4], filtered[:,4], 0.002)

In [52]:
nms_2.shape

torch.Size([238])

In [18]:
nms.shape

torch.Size([238])

In [20]:
det[:,:4]*1.3125

tensor([[1920.39917, 1934.51636, 1967.29089, 1984.18787],
        [1077.79346,  360.37067, 1123.25842,  408.52469],
        [1757.78149,  835.97528, 1804.74536,  883.40521],
        [1753.44934, 1851.46826, 1787.35620, 1887.73499],
        [1186.04248,  228.23886, 1223.33496,  267.55649],
        [1067.53662,  593.25043, 1109.82812,  639.22321],
        [1882.91309, 2928.81055, 1928.20496, 2978.40283],
        [1765.69824, 1742.82629, 1806.93884, 1787.05200],
        [ 987.20459,  235.19218, 1015.08563,  267.30124],
        [1508.77087,  974.53760, 1541.40808, 1009.21039],
        [1698.31812, 2261.82373, 1745.72803, 2312.69653],
        [1598.62537, 1647.95447, 1643.74414, 1693.85962],
        [ 904.70117,  771.58002,  940.29749,  807.78735],
        [1600.63879,  188.85822, 1635.09924,  226.59567],
        [1772.59631, 2064.13086, 1820.55225, 2113.82202],
        [2044.13940, 2957.20068, 2089.24390, 3002.25830],
        [1132.04114, 2804.63940, 1175.51489, 2849.33813],
        [1481.

In [3]:
def get_anchor():
    from models.experimental import attempt_load  # scoped to avoid circular import
    w = '/Users/zhenyu/Library/CloudStorage/Box-Box/MLProject:IphoneAOI/weights/yolov5l6_3072*2304_20211203/weights/best.pt'
    model_pt = torch.jit.load(w) if 'torchscript' in w else attempt_load(w)
    print('Anchors are:\n {}'.format(model_pt.model[-1].anchors))
    print(model_pt.model[-1].grid[0].shape)
    print(model_pt.model[-1].grid[1].shape)
    print(model_pt.model[-1].grid[2].shape)
    print(model_pt.model[-1].grid[3].shape)

In [4]:
get_anchor()

Fusing layers... 
Model Summary: 476 layers, 76126356 parameters, 0 gradients, 110.0 GFLOPs


Anchors are:
 tensor([[[ 2.37500,  3.37500],
         [ 5.50000,  5.00000],
         [ 4.75000, 11.75000]],

        [[ 6.00000,  4.25000],
         [ 5.37500,  9.50000],
         [11.25000,  8.56250]],

        [[ 4.37500,  9.40625],
         [ 9.46875,  8.25000],
         [ 7.43750, 16.93750]],

        [[ 6.81250,  9.60938],
         [11.54688,  5.93750],
         [14.45312, 12.37500]]])
torch.Size([1, 3, 392, 296, 2])
torch.Size([1, 3, 196, 148, 2])
torch.Size([1, 3, 98, 74, 2])
torch.Size([1, 3, 49, 37, 2])


In [52]:
get_anchor()

Fusing layers... 
Model Summary: 476 layers, 76126356 parameters, 0 gradients, 110.0 GFLOPs


Anchors are:
 tensor([[[ 2.37500,  3.37500],
         [ 5.50000,  5.00000],
         [ 4.75000, 11.75000]],

        [[ 6.00000,  4.25000],
         [ 5.37500,  9.50000],
         [11.25000,  8.56250]],

        [[ 4.37500,  9.40625],
         [ 9.46875,  8.25000],
         [ 7.43750, 16.93750]],

        [[ 6.81250,  9.60938],
         [11.54688,  5.93750],
         [14.45312, 12.37500]]])
torch.Size([1, 3, 392, 296, 2])
torch.Size([1, 3, 196, 148, 2])
torch.Size([1, 3, 98, 74, 2])
torch.Size([1, 3, 49, 37, 2])


In [6]:
float('3.141592653589793423634534795')

3.1415926535897936